<a target="_blank" href="https://colab.research.google.com/github/UpstageAI/cookbook/blob/main/cookbooks/upstage/Solar-Full-Stack LLM-101/05_3_OracleDB.ipynb">
<img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Retrieval Augmented Generation (RAG) Baseline


# Baseline

In [1]:
# set parameters

file = open("info/api.txt", "r")
api_key = file.read()
file.close()

file = open("info/datapath.txt", "r")
data_path = file.read()
file.close()

file = open("info/resultspath.txt", "r")
results_path = file.read()
file.close()

In [2]:
from langchain_upstage import UpstageEmbeddings

# 쿼리 전용 임베딩 모델
query_embeddings = UpstageEmbeddings(api_key=api_key, model="solar-embedding-1-large-query")

# 문장 전용 임베딩 모델
passage_embeddings = UpstageEmbeddings(api_key=api_key, model="solar-embedding-1-large-passage")

In [3]:
# funcion to extract an answer from response

import re

def extract_answer(response):
    """
    extracts the answer from the response using a regular expression.
    expected format: "[ANSWER]: (A) convolutional networks"

    if there are any answers formatted like the format, it returns None.
    """
    pattern = r"\[ANSWER\]:\s*\((A|B|C|D|E)\)"  # Regular expression to capture the answer letter and text
    match = re.search(pattern, response)

    if match:
        return match.group(1) # Extract the letter inside parentheses (e.g., A)
    else:
        return extract_again(response)

def extract_again(response):
    pattern = r"\b[A-J]\b(?!.*\b[A-J]\b)"
    match = re.search(pattern, response)
    if match:
        return match.group(0)
    else:
        return None

## Load DB

In [4]:
from langchain_upstage import UpstageLayoutAnalysisLoader
import os
import numpy as np


UPSTAGE_API_KEY = api_key

# .npy 파일 로드 (타입==넘파이)
ewhaDB = np.load(data_path+f'embedding/full_ewha500.npy')

ewhaDB_embed = np.load(data_path+f'embedding/full_ewha500_embed.npy')
ewhaDB_embed = ewhaDB_embed.tolist()

## DB Split (chunking)

## test set 가져와서 dataframe 만들기

In [5]:
# read samples.csv file
import pandas as pd

def read_data(data_path):
    data = pd.read_csv(data_path)
    prompts = data['prompts']
    answers = data['answers']
    # returns two lists: prompts and answers
    return prompts, answers

In [6]:
prompts, answers = read_data(os.path.join(data_path, 'test_own_ewha.csv'))

In [7]:
testdata = pd.read_csv(data_path+'test_own_ewha.csv')

In [ ]:
nowtest = pd.DataFrame(columns=['index', 'embed_ques', 'question', 'prompts', 'answers','predict', 'top1', 'top2', 'top3', 'top1plus', 'top2plus', 'top3plus', 'ground' ])

for index, row in testdata.iterrows():
    #if index % 100 != 0 : continue # 일단 실험할 땐 100개 단위로 끊어서 가져옴
    q = row.prompts
    a = row.answers
    question = q.partition('(A)')[0]
    question = question.partition(') ')[2]
    q = q.partition(') ')[2]
    try : 
        embedded_query = query_embeddings.embed_query(question) # >>질문<<만 받아와서 embedding 하기 // >>선지까지<< 받아와서 embedding하기
        nowtest.loc[len(nowtest)] = {'index':index, 'embed_ques' : embedded_query, 'question' : question, 'prompts' : q, 'answers' : a }

    except :
        print(f'pass: {index}')
        continue 


In [9]:
nowtest

,index,embed_ques,question,prompts,answers,predict,top1,top2,top3,top1plus,top2plus,top3plus,ground
0,0,"[-0.005329132080078125, 0.0017337799072265625,...",영어 및 정보 등에 관하여 일정한 기준의 능력이나 자격을 취득한 경우 인정 받는 학...,영어 및 정보 등에 관하여 일정한 기준의 능력이나 자격을 취득한 경우 인정 받는 학...,(D),None,None,None,None,NaN,NaN,NaN,NaN
1,1,"[-0.00460052490234375, -0.0174713134765625, -0...",각 대학에 따른 학위의 종류로 해당하지 않는 것은 무엇인가?\n,각 대학에 따른 학위의 종류로 해당하지 않는 것은 무엇인가?\n(A) 이학사\n(B...,(E),None,None,None,None,NaN,NaN,NaN,NaN
2,2,"[0.007404327392578125, -0.0216522216796875, 0....",부칙에 학과명 변경 및 폐과된 학과는 무엇에 의해 언제까지 존속하는가?\n,부칙에 학과명 변경 및 폐과된 학과는 무엇에 의해 언제까지 존속하는가?\n(A) 현...,(B),None,None,None,None,NaN,NaN,NaN,NaN
3,3,"[-0.002162933349609375, -0.039215087890625, -0...",학생 포털에서 본인의 성적을 조회할 수 있는 시기는 언제입니까?\n,학생 포털에서 본인의 성적을 조회할 수 있는 시기는 언제입니까?\n(A) 시험 직후...,(B),None,None,None,None,NaN,NaN,NaN,NaN
4,4,"[0.0089111328125, -0.0259552001953125, -0.0064...",학칙 제1조(목적)에 따르면 이화여자대학교의 설립 목적에 해당하지 않는 것은 무엇입...,학칙 제1조(목적)에 따르면 이화여자대학교의 설립 목적에 해당하지 않는 것은 무엇입...,(C),None,None,None,None,NaN,NaN,NaN,NaN
5,5,"[0.0015554428100585938, -0.019683837890625, -0...",학칙 제3조에서 명시된 본교의 주소에 포함되지 않는 요소는 무엇입니까?\n,학칙 제3조에서 명시된 본교의 주소에 포함되지 않는 요소는 무엇입니까?\n(A) 서...,(D),None,None,None,None,NaN,NaN,NaN,NaN
6,6,"[-0.018157958984375, -0.0157318115234375, -0.0...",학칙 제12조(휴업일)에서 임시휴업을 결정할 수 있는 근거로 인정되지 않는 경우는?\n,학칙 제12조(휴업일)에서 임시휴업을 결정할 수 있는 근거로 인정되지 않는 경우는?...,(C),None,None,None,None,NaN,NaN,NaN,NaN
7,7,"[0.0029163360595703125, -0.018707275390625, -0...",학칙 제23조(수업연한)에 따르면 건축학과와 약학대학의 수업연한이 다른 이유로 올바...,학칙 제23조(수업연한)에 따르면 건축학과와 약학대학의 수업연한이 다른 이유로 올바...,(D),None,None,None,None,NaN,NaN,NaN,NaN
8,8,"[-5.4895877838134766e-05, -0.01477813720703125...",학칙 제26조(휴학)에서 규정된 휴학의 최대 연장 가능 기간을 초과하는 경우가 아닌...,학칙 제26조(휴학)에서 규정된 휴학의 최대 연장 가능 기간을 초과하는 경우가 아닌...,(C),None,None,None,None,NaN,NaN,NaN,NaN
9,9,"[-0.002269744873046875, 0.0017795562744140625,...",학칙 제50조(졸업)에서 복수전공 이수자가 요구되는 추가 조건은 무엇입니까?\n,학칙 제50조(졸업)에서 복수전공 이수자가 요구되는 추가 조건은 무엇입니까?\n(A...,(B),None,None,None,None,NaN,NaN,NaN,NaN


## chunk마다 Evidence와 Reasoning 생성

계산 효율을 위해 그냥 모든 chunk를 미리 segment+(note) 해두기 (나중에 유사도 계산 후에 관련있는 애들 다시 중복으로 계산 할 바에 미리 해둔다는 뜻)

In [18]:
############# Segment+ first prompt ( Evidence and Reasoning ) ##########

### 논문에서 제공하는 영어 버전으로 하면, 답변에 영어가 섞이는 현상 -> 프롬프트를 한글로 바꿈
### 논문 그대로 프롬프트를 제공하면 전부 이해하지 못하는 것 같아서 좀 간소화시킴

from langchain_core.prompts import PromptTemplate
from langchain_upstage import ChatUpstage

 
llm = ChatUpstage(api_key = api_key)

prompt_template = PromptTemplate.from_template(
    '''

    You are provided with a segment from a long document along with a question related to this document.
    Segment Content: {context1}
    —
    Question: {question}
    —

    Your task: Evaluate the provided segment against the question to identify and categorize information into two distinct types: 'Evidence' and 'Reasoning'. Your assessment and categorization should adhere to the following guidelines:
    Guidelines for Note-Writing:
    Your note should be meticulously structured into two main parts: Evidence and Reasoning,
    following these guidelines:
    - Evidence:
        1. Extract key sentences or descriptions from the segment that are pertinent to the question, with a focus on specific details such as
        numbers, relevant words, and other significant
        elements.
          (1) Include content that directly relates to the question, providing a straightforward answer.
          (2) Also include content that may not directly answer the question but is valuable for answering it when combined with information from other segments. For instance, for questions about someone’s birthplace, include all mentioned birthplaces for potential matching in later analysis. Similarly, if the question involves several events but this segment only contains information about one event, you should include it.
        2. Accurately quote the directly related sentences to present clear and unambiguous evidence.
    - Reasoning:
        1. Analyze the question and any subquestions, offering answers, summaries, interpretations, or any relevant commentary to deepen the understanding of the question.
    
    The note should be formatted in JSON as follows:
    [ 'Evidence': 'Your evidence content here',
    'Reasoning': 'Your reasoning content here' ]
            
    '''

)
segplus1 = prompt_template | llm


In [11]:
nowtest.question[0]

'영어 및 정보 등에 관하여 일정한 기준의 능력이나 자격을 취득한 경우 인정 받는 학점은 몇점인가?\n'

## question - chunk 유사도 계산

In [19]:
# 유사도 높은 chunk top3를 df에 넣어두기

import numpy as np


for idx, row in nowtest.iterrows() : # 질문 받아오기 
    #if idx == 10 : break ## 테스트용
    embed_ques= row.embed_ques
    prompt = row.prompts
    ques = row.question

    sorted_idx = (np.array(embed_ques) @ np.array(ewhaDB_embed).T).argsort()[::-1]

    print( f" ==================={idx+1}==================== ")
    for topidx in range(3) :
        top = f"top{topidx+1}"
        topplus = f"top{topidx+1}plus"

        nowtest.loc[idx, top] = ewhaDB[sorted_idx[topidx]]

        response = segplus1.invoke({"context1": ewhaDB[sorted_idx[topidx]], "question": prompt})
        nowtest.loc[idx, topplus] = response.content
        print(response.content) # 확인용
        print("***")
    


 ===================1==================== 


/tmp/ipykernel_448111/1955669393.py:22: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '{
  "Evidence": "외에 학점을 취득할 수 있으며,그 학점수는 12학점을 초과할 수 없다.",
  "Reasoning": "The given segment does not provide information about the number of credits awarded for acquiring a certain level of proficiency or certification in English and information. The segment discusses the maximum number of credits that can be earned through special examinations and the recognition of credits from other institutions or pre-admission courses. It does not mention anything specific to the question about the number of credits received for acquiring proficiency or certification in English and information. Therefore, the answer to the question cannot be determined from the given segment."
}' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  nowtest.loc[idx, topplus] = response.content


{
  "Evidence": "외에 학점을 취득할 수 있으며,그 학점수는 12학점을 초과할 수 없다.",
  "Reasoning": "The given segment does not provide information about the number of credits awarded for acquiring a certain level of proficiency or certification in English and information. The segment discusses the maximum number of credits that can be earned through special examinations and the recognition of credits from other institutions or pre-admission courses. It does not mention anything specific to the question about the number of credits received for acquiring proficiency or certification in English and information. Therefore, the answer to the question cannot be determined from the given segment."
}
***


/tmp/ipykernel_448111/1955669393.py:22: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '{
  "Evidence": [
    "제35조의2(학점취득의 특례) ① 교양과목과 전공기초과목 중 특정교과목은 이를 수강하지 아니하여도 학점취득특별시험에 의하여 학기당 최대취득학점 외에 학점을 취득할 수 있으며,그 학점수는 12학점을 초과할 수 없다. (개정 1996.2)"
  ],
  "Reasoning": [
    "The segment provides information about special examinations for acquiring credits for specific subjects in liberal arts and major foundation subjects. However, it does not specify the number of credits that can be recognized for obtaining a certain standard of ability or qualifications in English and information. Therefore, we cannot find a direct answer to the question in the segment."
  ]
}' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  nowtest.loc[idx, topplus] = response.content


{
  "Evidence": [
    "제35조의2(학점취득의 특례) ① 교양과목과 전공기초과목 중 특정교과목은 이를 수강하지 아니하여도 학점취득특별시험에 의하여 학기당 최대취득학점 외에 학점을 취득할 수 있으며,그 학점수는 12학점을 초과할 수 없다. (개정 1996.2)"
  ],
  "Reasoning": [
    "The segment provides information about special examinations for acquiring credits for specific subjects in liberal arts and major foundation subjects. However, it does not specify the number of credits that can be recognized for obtaining a certain standard of ability or qualifications in English and information. Therefore, we cannot find a direct answer to the question in the segment."
  ]
}
***


/tmp/ipykernel_448111/1955669393.py:22: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[
  "Evidence": "제31조(재입학) ① 제28조제1호 부터 제4호까지에 따라 제적된 자 및 제30조에 의하여 자퇴한 자에게는 소속 대학장의 제청에 의하여 총장이 그 정상을 참작하여 정원의 결원이 있을 때에 재입학을 허가할 수 있다. (개정 2008.8.19., 2017.8.16.)",
  "Reasoning": "The segment provided does not contain any information about the number of credits recognized for obtaining a certain level of ability or qualification in English and information. Therefore, none of the answer options (A), (B), (C), or (D) can be determined from this segment. The segment discusses re-admission after expulsion or dropping out, and there is no direct relation to the question regarding credits for English and information proficiency."
]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  nowtest.loc[idx, topplus] = response.content


[
  "Evidence": "제31조(재입학) ① 제28조제1호 부터 제4호까지에 따라 제적된 자 및 제30조에 의하여 자퇴한 자에게는 소속 대학장의 제청에 의하여 총장이 그 정상을 참작하여 정원의 결원이 있을 때에 재입학을 허가할 수 있다. (개정 2008.8.19., 2017.8.16.)",
  "Reasoning": "The segment provided does not contain any information about the number of credits recognized for obtaining a certain level of ability or qualification in English and information. Therefore, none of the answer options (A), (B), (C), or (D) can be determined from this segment. The segment discusses re-admission after expulsion or dropping out, and there is no direct relation to the question regarding credits for English and information proficiency."
]
***
 ===================2==================== 
{
  "Evidence": "입학(편입학 및 재입학 포함)은 해당 대학장의 제청으로 총장이 허가한다. 다만, 제15조제1항의 단서의 경우나 특별전형, 기타 입학에 관하여 긴급한 필요가 있는 이화여자대학교 학칙 경우에는 대학장의 제청을 거치지 아니할 수 있다. (개정 2015. 2.6.)",
  "Reasoning": "이 문장은 각 대학에 따른 학위의 종류에 대한 정보를 제공하지 않습니다. 대신, 입학 절차와 관련된 내용을 다루고 있으며, 학위의 종류와는 직접적인 연관이 없습니다. 이 문장은 학위 종류에 대한 질문에 대한 직접적인 답변을 제공하지 않으므로, 

In [20]:
nowtest.top3[1]

'외에 학점을 취득할 수 있으며,그 학점수는 12학점을 초과할 수 없다. (개정 1996.2.15., 2017.2.8.)\n② 학점취득특별시험의 방법, 종류에 관한 세부 사항은 총장이 따로 정한다. (개정\n2001.9.24)제35조의3(학점의 인정) ① 다음 각 호의 경우에 취득한 학점은 총장의 승인을 얻어 졸업에\n필요한 학점의 2분의 1의 범위 안에서 이를 본교에서 취득한 것으로 본다. (개정\n2012.12.31.)1. 재학 중 국내외의 다른 학교에서 학점을 취득한 경우\n2. 입학 전 국내외의 고등학교와 「고등교육법」 제2조 각 호의 학교에서 대학교육과정에 상\n당하는 교과목을 이수한 경우\n② 학점의 인정범위 등 제1항의 시행에 필요한 사항은 총장이 따로 정한다. (개정\n1998.6.23)③ 삭제 (1998.6.23.)제35조의4(언어교육원 수강과목 학점의 인정) 입학 전 본교 언어교육원에서 소정의 과정을 이\n수한 경우 입학 후 대학의 학점으로 인정할 수 있다. 학점인정대상자 및 범위'

In [21]:
nowtest

,index,embed_ques,question,prompts,answers,predict,top1,top2,top3,top1plus,top2plus,top3plus,ground
0,0,"[-0.005329132080078125, 0.0017337799072265625,...",영어 및 정보 등에 관하여 일정한 기준의 능력이나 자격을 취득한 경우 인정 받는 학...,영어 및 정보 등에 관하여 일정한 기준의 능력이나 자격을 취득한 경우 인정 받는 학...,(D),None,"외에 학점을 취득할 수 있으며,그 학점수는 12학점을 초과할 수 없다. (개정 19...","5.6.17.]제9장 교육과정, 이수, 수료 및 졸업 (개정 2015.2.6.)제3...",4)제29조 삭제 (1987.9.1)\n제30조(자퇴) 자퇴하고자 하는 자는 보증인...,"{\n ""Evidence"": ""외에 학점을 취득할 수 있으며,그 학점수는 12학점...","{\n ""Evidence"": [\n ""제35조의2(학점취득의 특례) ① 교양...","[\n ""Evidence"": ""제31조(재입학) ① 제28조제1호 부터 제4호까지...",NaN
1,1,"[-0.00460052490234375, -0.0174713134765625, -0...",각 대학에 따른 학위의 종류로 해당하지 않는 것은 무엇인가?\n,각 대학에 따른 학위의 종류로 해당하지 않는 것은 무엇인가?\n(A) 이학사\n(B...,(E),None,당하게 할 수 있다.② 입학사정관의 조직과 운영에 관한 사항은 총장이 따로 정한다....,"5.6.17.]제9장 교육과정, 이수, 수료 및 졸업 (개정 2015.2.6.)제3...","외에 학점을 취득할 수 있으며,그 학점수는 12학점을 초과할 수 없다. (개정 19...","{\n ""Evidence"": ""입학(편입학 및 재입학 포함)은 해당 대학장의 제청...","{\n ""Evidence"": ""제32조(교과목의 분류) 본교의 교과는 교양과목, ...","{\n ""Evidence"": [\n ""외에 학점을 취득할 수 있으며,그 학점...",NaN
2,2,"[0.007404327392578125, -0.0216522216796875, 0....",부칙에 학과명 변경 및 폐과된 학과는 무엇에 의해 언제까지 존속하는가?\n,부칙에 학과명 변경 및 폐과된 학과는 무엇에 의해 언제까지 존속하는가?\n(A) 현...,(B),None,.21)\n⑥ 휴학기간은 재학연한에 산입하지 아니한다. (개정 2014.11.21)...,있다. (개정\n2000.2.1)\n③ 삭제 (1984.3.5)\n④ 수강신청에 관...,4)제29조 삭제 (1987.9.1)\n제30조(자퇴) 자퇴하고자 하는 자는 보증인...,"{\n ""Evidence"": [\n ""제7장 교내전학제25조(전과) ① 학생...",[\n 'Evidence': '제6조(학과명칭 및 모집단위변경) ①학칙 제2조의 ...,"{\n ""Evidence"": [\n ""제31조(재입학) ① 제28조제1호 부...",NaN
3,3,"[-0.002162933349609375, -0.039215087890625, -0...",학생 포털에서 본인의 성적을 조회할 수 있는 시기는 언제입니까?\n,학생 포털에서 본인의 성적을 조회할 수 있는 시기는 언제입니까?\n(A) 시험 직후...,(B),None,허가된 자는 정해진 기일 내에 입학금 기타 납입금을 납입하여\n야 한다. (개정 1...,4)제29조 삭제 (1987.9.1)\n제30조(자퇴) 자퇴하고자 하는 자는 보증인...,.21)\n⑥ 휴학기간은 재학연한에 산입하지 아니한다. (개정 2014.11.21)...,"{\n ""Evidence"": [\n ""학생은 매 학기초 소정 기일 내에 소정...","[\n ""Evidence"": ""제30조(자퇴) 자퇴하고자 하는 자는 보증인이 연서...","[\n ""Evidence"": ""제26조(휴학) ① 질병 기타 부득이한 사정으로 3...",NaN
4,4,"[0.0089111328125, -0.0259552001953125, -0.0064...",학칙 제1조(목적)에 따르면 이화여자대학교의 설립 목적에 해당하지 않는 것은 무엇입...,학칙 제1조(목적)에 따르면 이화여자대학교의 설립 목적에 해당하지 않는 것은 무엇입...,(C),None,. (신설 2015.9.18.)이화여자대학교 학칙⑧ 제4항에서 정한 휴학기간이 만료...,당하게 할 수 있다.② 입학사정관의 조직과 운영에 관한 사항은 총장이 따로 정한다....,"5.6.17.]제9장 교육과정, 이수, 수료 및 졸업 (개정 2015.2.6.)제3...","{\n ""Evidence"": [\n ""이화여자대학교 학칙⑧ 제4항에서 정한 ...","{\n ""Evidence"": ""학칙 제1조(목적)에 따르면 이화여자대학교의 설립 ...","{\n ""Evidence"": ""제9장 교육과정, 이수, 수료 및 졸업 (개정 20...",NaN
5,5,"[0.0015554428100585938, -0.019683837890625, -0...",학칙 제3조에서 명시된 본교의 주소에 포함되지 않는 요소는 무엇입니까?\n,학칙 제3조에서 명시된 본교의 주소에 포함되지 않는 요소는 무엇입니까?\n(A) 서...,(D),None,4)제29조 삭제 (1987.9.1)\n제30조(자퇴) 자퇴하고자 하는 자는 보증인...,당하게 할 수 있다.② 입학사정관의 조직과 운영에 관한 사항은 총장이 따로 정한다....,.21)\n⑥ 휴학기간은 재학연한에 산입하지 아니한다. (개정 2014.11.21)...,"{\n ""Evidence"": [\n ""제29조 삭제 (1987.9.1)"",\...","{\n ""Evidence"": [\n ""당하게 할 수 있다.② 입학사정관의 조...","{\n ""Evidence"": [\n {\n ""text"": ""제8장 ...",NaN
6,6,"[-0.018157958984375, -0.0157318115234375, -0.0...",학칙 제12조(휴업일)에서 임시휴업을 결정할 수 있는 근거로 인정되지 않는 경우는?\n,학칙 제12조(휴업일)에서 임시휴업을 결정할 수 있는 근거로 인정되지 않는 경우는?...,(C),None,.21)\n⑥ 휴학기간은 재학연한에 산입하지 아니한다. (개정 2014.11.21)...,"988.7.28)\n③ 1회의 휴학기간은 1년 이내로 한다. 다만, 교과과정상의 필...",. (신설 2015.9.18.)이화여자대학교 학칙⑧ 제4항에서 정한 휴학기간이 만료...,"{\n ""Evidence"": [\n ""제26조(휴학) ① 질병 기타 부득이한...","{\n ""Evidence"": [\n ""④ 휴학기간은 통산하여 3년(건축학전공...","{\n ""Evidence"": [\n ""제4항에서 정한 휴학기간이 만료되었으나...",NaN
7,7,"[0.0029163360595703125, -0.018707275390625, -0...",학칙 제23조(수업연한)에 따르면 건축학과와 약학대학의 수업연한이 다른 이유로 올바...,학칙 제23조(수업연한)에 따르면 건축학과와 약학대학의 수업연한이 다른 이유로 올바...,(D),None,있다. (개정\n2000.2.1)\n③ 삭제 (1984.3.5)\n④ 수강신청에 관...,"없다. (개정 2014.11.21)② 의과대학 학생의 재학연한은 의예과는 4년, 의...","988.7.28)\n③ 1회의 휴학기간은 1년 이내로 한다. 다만, 교과과정상의 필...","{\n""Evidence"": ""제23조(수업연한) ① 각 대학의 수업연한은 4년으로 ...","{\n ""Evidence"": [\n ""의과대학 학생의 재학연한은 의예과는 4...","{\n ""Evidence"": [\n ""③ 1회의 휴학기간은 1년 이내로 한다...",NaN
8,8,"[-5.4895877838134766e-05, -0.01477813720703125...",학칙 제26조(휴학)에서 규정된 휴학의 최대 연장 가능 기간을 초과하는 경우가 아닌...,학칙 제26조(휴학)에서 규정된 휴학의

In [ ]:
############# Segment+ 2nd Prompt ##########

from langchain_core.prompts import PromptTemplate
from langchain_upstage import ChatUpstage


llm = ChatUpstage(api_key = api_key)

prompt_template = PromptTemplate.from_template(
    '''
    
    You are an assistant with expertise in Ewha University policies and history. Use the provided context to answer accurately.
    Let's think step by step.

    Context: {context1} {context2} {context3}

    Question: {question}

    Final Answer:
    
    '''

)
segplus2 = prompt_template | llm


responses = []
for idx, row in nowtest.iterrows() : 
        
    response = segplus2.invoke({"question": row.prompts, "context1": row.top1plus, "context2": row.top2plus, "context3": row.top3plus}) # 선지 전까지 받아오기
    responses.append(response.content)
    nowtest.loc[idx, 'predict'] = response.content

responses

['D',
 '(C) 공학사',
 '(B) 구 학칙에 의하여 재학생이 졸업하는 연도까지 존속한다',
 '(B) 성적 입력 마감 후',
 '(C) 지역사회의 봉사',
 '(D) 강남구입니다.',
 '(C) 학생총회 요청',
 'B',
 '(E) 중대한 질병으로 인한 휴학: 총장의 판단에 따라 연장 가능',
 '(B) 전공별 종합시험 합격',
 'E',
 'C',
 '(D) 타교에 입학한 경우',
 'D',
 '(A)',
 '(C) 15시간 이상',
 '(E) 특정 종교 교리 확립',
 '정답은 (C) 전공 결정은 일정 학점 이수 후 이루어진다입니다.',
 '질문에 대한 정답은 (D) 엘텍공과대학 건축학전공입니다. 주어진 학칙에 따르면, 엘텍공과대학의 건축학전공은 수업 연한이 4년이 아닌 5년으로 규정되어 있습니다.',
 '(C) 21학점',
 '(C) 학기 중간에 전과를 신청한 경우',
 '(C) 동기방학',
 '(B)',
 '답변:\nC',
 '(B) 시험 부정행위']

## check accuracy

In [ ]:
######### 정답 확인 + wrong 뽑아내기 ######

# print accuracy

cnt = 0
wrong2 = []
for idx, (answer, response) in enumerate(zip(answers, nowtest['predict'])):
    print("-"*10)
    generated_answer = extract_answer(response)
    print(response)
    # check
    if generated_answer:
        print(f"idx: {idx} | generated answer: {generated_answer}, answer: {answer}")
    else:
        print("extraction fail")

    if generated_answer == None:
        wrong2.append(idx) 
        continue
    
    if generated_answer in answer:
        cnt += 1
    else : 
        wrong2.append(idx)
        

print()
print(f"acc: {(cnt/len(answers))*100}%")
print()
print("wrong:",wrong2)

----------
D
idx: 0 | generated answer: D, answer: (D)
----------
(C) 공학사
idx: 1 | generated answer: C, answer: (E)
----------
(B) 구 학칙에 의하여 재학생이 졸업하는 연도까지 존속한다
idx: 2 | generated answer: B, answer: (B)
----------
(B) 성적 입력 마감 후
idx: 3 | generated answer: B, answer: (B)
----------
(C) 지역사회의 봉사
idx: 4 | generated answer: C, answer: (C)
----------
(D) 강남구입니다.
idx: 5 | generated answer: D, answer:  (D)
----------
(C) 학생총회 요청
idx: 6 | generated answer: C, answer:  (C)
----------
B
idx: 7 | generated answer: B, answer:  (D)
----------
(E) 중대한 질병으로 인한 휴학: 총장의 판단에 따라 연장 가능
idx: 8 | generated answer: E, answer:  (C)
----------
(B) 전공별 종합시험 합격
idx: 9 | generated answer: B, answer:  (B)
----------
E
idx: 10 | generated answer: E, answer:  (B)
----------
C
idx: 11 | generated answer: C, answer:  (C)
----------
(D) 타교에 입학한 경우
idx: 12 | generated answer: D, answer:  (B)
----------
D
idx: 13 | generated answer: D, answer:  (E)
----------
(A)
idx: 14 | generated answer: A, answer:  (A)
----------
(C)

## groundedness check

### 질문할 때마다 답이 달라져서 안 할 거임 

In [ ]:
import os
from langchain_upstage import UpstageGroundednessCheck # langchain_upstage==0.1.3
 
groundedness_check = UpstageGroundednessCheck()
 
request_input = {
    "context": f"Mauna Kea is an inactive volcano on the island of Hawai'i. Its peak is 4,207.3 m above sea level, making it the highest point in Hawaii and second-highest peak of an island on Earth.",
    "answer": "Mauna Kea is 5,207.3 meters tall.",
}
response = groundedness_check.invoke(request_input)
print(response)